In [1]:
import requests
import json
import pandas as pd
import time
import logging
import random
import os


In [2]:
url = "https://apionline.homedepot.com/federation-gateway/graphql?opname=productClientOnlyProduct"

In [3]:
with open('headers.json', 'r') as file:
    headers = json.load(file)
with open('payload.json', 'r') as file:
    payload = json.load(file)
# 发送 POST 请求
response = requests.post(url, json=payload, headers=headers)
payload['variables']['itemId'] = "331622362"
response = requests.post(url, json=payload, headers=headers)
print(response)

<Response [206]>


In [4]:
response.text

'{"data":{"product":{"dataSources":"catalog","identifiers":{"productType":"MERCHANDISE","storeSkuNumber":"1012277445","brandName":"CARRO","productLabel":"Louy 52 in. Color Changing Integrated LED Indoor Matte White 10-Speed DC Ceiling Fan with Light Kit and Wall Control","canonicalUrl":"/p/CARRO-Louy-52-in-Color-Changing-Integrated-LED-Indoor-Matte-White-10-Speed-DC-Ceiling-Fan-with-Light-Kit-and-Wall-Control-HCFW523E5-L12-WP-1/331622362","itemId":"331622362","isSuperSku":true,"parentId":"331621844","sampleId":null,"skuClassification":"normal","modelNumber":"HCFW523E5-L12-WP-1","specialOrderSku":null,"toolRentalSkuNumber":null,"rentalCategory":null,"rentalSubCategory":null,"upc":"810154055821","upcGtin13":"0810154055821","brandMainPageUrl":"/b/CARRO/N-5yc1vZrck","roomVOEnabled":false,"__typename":"Identifiers"},"availabilityType":{"type":"Shared","discontinued":false,"status":true,"buyable":true,"__typename":"AvailabilityType"},"media":{"augmentedRealityLink":null,"images":[{"url":"htt

In [58]:
extract_data(response)

{'storeSkuNumber': '1012277445',
 'brandName': 'CARRO',
 'productLabel': 'Louy 52 in. Color Changing Integrated LED Indoor Matte White 10-Speed DC Ceiling Fan with Light Kit and Wall Control',
 'canonicalUrl': '/p/CARRO-Louy-52-in-Color-Changing-Integrated-LED-Indoor-Matte-White-10-Speed-DC-Ceiling-Fan-with-Light-Kit-and-Wall-Control-HCFW523E5-L12-WP-1/331622362',
 'itemId': '331622362',
 'parentId': '331621844',
 'modelNumber': 'HCFW523E5-L12-WP-1',
 'value': 137.26,
 'original': None,
 'inventory': 86,
 'out_of_stock_status': True,
 'review_num': None,
 'rating': None}

In [6]:
headers

{'accept': '*/*',
 'accept-encoding': 'gzip, deflate, br, zstd',
 'accept-language': 'en-US,en;q=0.9',
 'content-type': 'application/json',
 'cookie': 'QuantumMetricUserID=a2899551da75d63cbb97689ca698cdc7; HD_DC=origin; at_check=true; AMCVS_F6421253512D2C100A490D45%40AdobeOrg=1; THD_PERSIST=; THD_CACHE_NAV_PERSIST=; DELIVERY_ZIP_TYPE=DEFAULT; thda.s=7fe92dab-a91d-ce53-fa91-8a2c1a312ad2; thda.u=663cb1bf-11fb-6c11-8ac5-2e9366add3ad; _px_f394gi7Fvmc43dfg_user_id=YWQyYzZhZDEtMzNmOC0xMWVmLTliMzEtM2IyY2ZiYWFiMGY0; ajs_anonymous_id=d7861241-60b9-44eb-9d5a-fbbddaa73695; thda.m=76721570817405704221197643616351372219; aam_uuid=76470141564458929311223218151539404238; trx=6582811316921143341; _gcl_au=1.1.17785004.1719432867; THDSSO=GydXe60SiNG9BDilB9EeXKpqD6YLNATbogzGIqEAdTHUd55O7mLzJFpQOnoUm6uI2D2OTd5dGca3YXcLQWtaDAJ5iwM1AzVcDuWQ2QZOibJpNKLkINS8biHtMrqvtmn2GgO2CrzYGJu2um5AiMTXYJpzNjGivyh0VUZuQluZ9VdDAFUHHRK2nc89jM0iUfXO21cHurBgddK3Uh0Yd3rHUHgavHzABJ9YExUU8OT3eODhZkME3pbEaWl8lqnxetZY5bUz4Qpoh9bgz7

In [4]:
response.json()

{'data': {'productClientOnlyProduct': None},
 'error': [{'message': 'productClientOnlyProduct errors'}]}

In [56]:
def extract_data(response):
    try:
        data = response.json().get('data', {})
        
        identifiers = data.get('product', {}).get('identifiers', {})
        fulfillment_options = data.get('product', {}).get('fulfillment', {}).get('fulfillmentOptions', [])
        pricing = data.get('product', {}).get('pricing', {})
        reviews = data.get('product', {}).get('reviews', {})
        
        # Safely extract locations if available
        locations = {}
        if fulfillment_options and 'services' in fulfillment_options[0]:
            locations = fulfillment_options[0]['services'][0].get('locations', [{}])[0]

        # Handle missing fields with defaults
        filtered_locations = {
            'inventory': locations.get('inventory', {}).get('quantity', None),
            'out_of_stock_status': locations.get('inventory', {}).get('isInStock', None)
        }
        filtered_pricing = {
            'value': pricing.get('value', None),
            'original': pricing.get('original', None)
        }

        filtered_identifiers = {
            'storeSkuNumber': identifiers.get('storeSkuNumber', None),
            'brandName': identifiers.get('brandName', None),
            'productLabel': identifiers.get('productLabel', None),
            'canonicalUrl': identifiers.get('canonicalUrl', None),
            'itemId': identifiers.get('itemId', None),
            'parentId': identifiers.get('parentId', None),
            'modelNumber': identifiers.get('modelNumber', None)
        }
        if reviews is not None:
            filtered_reviews = {
                'review_num': reviews.get('ratingsReviews', {}).get('totalReviews', None),
                'rating': reviews.get('ratingsReviews', {}).get('averageRating', None)
            }
        else: 
            filtered_reviews = {
            'review_num': None,
            'rating': None
            }

        # Combine all filtered data into a single result
        results = {**filtered_identifiers, **filtered_pricing, **filtered_locations, **filtered_reviews}

    except Exception as e:
        # Log the error and return empty result if structure is not as expected
        logging.error(f"Error extracting data: {e}")
        results = {
            'storeSkuNumber': None,
            'brandName': None,
            'productLabel': None,
            'canonicalUrl': None,
            'itemId': None,
            'parentId': None,
            'modelNumber': None,
            'inventory': None,
            'out_of_stock_status': None,
            'value': None,
            'original': None,
            'review_num': None,
            'rating': None
        }

    return results
    

In [8]:
# Setting up logging for error tracking
logging.basicConfig(level=logging.INFO)

In [9]:
df = pd.read_excel(r"C:\Users\cs\OneDrive\Desktop\carro\redfin\Carro-sku.xlsx", sheet_name="每月一次爬取Carro产品信息")

In [10]:
def save_checkpoint(results, file_path='checkpoint.json'):
    with open(file_path, 'w') as f:
        json.dump(results, f, indent=4)

In [11]:
def load_checkpoint(file_path='checkpoint.json'):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        return []

In [12]:
def update_cookie(headers, new_cookie):
    headers['Cookie'] = new_cookie
    return headers

In [76]:
def hd_scraper(df, headers, payload):
    url = "https://apionline.homedepot.com/federation-gateway/graphql?opname=productClientOnlyProduct"
    results = load_checkpoint()
    processed_ids = {item['itemId'] for item in results}  # 已处理的itemId集合
    session = requests.Session()

    assert '<ID>' in df.columns, "Error: '<ID>' column not found in the Excel sheet."

    for item_id in df['<ID>']:
        if str(item_id) in processed_ids:
            continue  # 跳过已处理的itemId
        
        payload['variables']['itemId'] = item_id
        print(f"现在处于{item_id}")
        time.sleep(15)
        try:
            response = session.post(url, json=payload, headers=headers)

            # 检查cookie失效导致的状态码206
            if response.status_code == 206:
                logging.warning(f"Cookie expired for item {item_id}. Updating cookie...")
                break
                # headers = update_cookie(headers, new_cookie)
                # response = session.post(url, json=payload, headers=headers)

            # 请求成功
            if response.status_code == 200:
                result = extract_data(response)
                if all(value is None for value in result.values()):
                    result['itemId'] = str(item_id)
                results.append(result)
                save_checkpoint(results)  # 每次请求成功后保存checkpoint
                
            else:
                logging.error(f"Failed request for item {item_id}. Status code: {response.status_code}")
        
        except Exception as e:
            logging.error(f"Error occurred for item {item_id}: {e}")

    logging.info(f"Scraping completed. Total results: {len(results)}")
    return results

In [80]:
with open('headers.json', 'r') as file:
    headers = json.load(file)
with open('payload.json', 'r') as file:
    payload = json.load(file)

In [82]:
hd_scraper(df, headers, payload)

现在处于314443230
现在处于322715063
现在处于314443217
现在处于314443284
现在处于314443243
现在处于314443255
现在处于314443269
现在处于322645334


ERROR:root:Error extracting data: 'NoneType' object has no attribute 'get'


现在处于314443275


ERROR:root:Error extracting data: 'NoneType' object has no attribute 'get'


现在处于314443281
现在处于314443242
现在处于317332543
现在处于314443264
现在处于314443236
现在处于314443244
现在处于314443239
现在处于314443232
现在处于314443277
现在处于314443283
现在处于322644848
现在处于315652215
现在处于315652252


ERROR:root:Error extracting data: 'NoneType' object has no attribute 'get'


现在处于315651618
现在处于315643876
现在处于323684397
现在处于315277027
现在处于321806221


INFO:root:Scraping completed. Total results: 608


[{'storeSkuNumber': '1006988826',
  'brandName': 'CARRO',
  'productLabel': 'Veter 56 in. Dimmable LED Indoor/Outdoor White Smart Ceiling Fan with Light and Remote, Works with Alexa/Google Home',
  'canonicalUrl': '/p/CARRO-Veter-56-in-Dimmable-LED-Indoor-Outdoor-White-Smart-Ceiling-Fan-with-Light-and-Remote-Works-with-Alexa-Google-Home-HS563B3-L22-W1-1/318938653',
  'itemId': '318938653',
  'parentId': '318938472',
  'modelNumber': 'HS563B3-L22-W1-1',
  'value': 169.0,
  'original': 289.0,
  'inventory': 103,
  'out_of_stock_status': True,
  'review_num': '33',
  'rating': '3.8182'},
 {'storeSkuNumber': '1006988827',
  'brandName': 'CARRO',
  'productLabel': 'Veter 56 in. Dimmable LED Indoor/Outdoor Black Smart Ceiling Fan with Light and Remote, Works with Alexa/Google Home',
  'canonicalUrl': '/p/CARRO-Veter-56-in-Dimmable-LED-Indoor-Outdoor-Black-Smart-Ceiling-Fan-with-Light-and-Remote-Works-with-Alexa-Google-Home-HS563B3-L22-B2-1G/318938681',
  'itemId': '318938681',
  'parentId': 

In [83]:
results = load_checkpoint()

In [84]:
len(results)

608

In [88]:
df = df[['Number', '<ID>', 'MFG Model #']]
len(df)

607

In [101]:
df_json = pd.DataFrame(results)

In [97]:
df_merge = pd.merge(df, df_json, how='left', left_on='<ID>', right_on='itemId')

In [98]:
df_merge

,Number,<ID>,MFG Model #,storeSkuNumber,brandName,productLabel,canonicalUrl,itemId,parentId,modelNumber,value,original,inventory,out_of_stock_status,review_num,rating
0,1,318938653,HS563B3-L22-W1-1,1006988826,CARRO,Veter 56 in. Dimmable LED Indoor/Outdoor White...,/p/CARRO-Veter-56-in-Dimmable-LED-Indoor-Outdo...,318938653,318938472,HS563B3-L22-W1-1,169.00,289.0,103.0,True,33,3.8182
1,2,318938681,HS563B3-L22-B2-1G,1006988827,CARRO,Veter 56 in. Dimmable LED Indoor/Outdoor Black...,/p/CARRO-Veter-56-in-Dimmable-LED-Indoor-Outdo...,318938681,318938472,HS563B3-L22-B2-1G,169.00,289.0,190.0,True,33,3.8182
2,3,318630540,S525J1-L11-B2-1G,1006880643,CARRO,Aspen 52 in. Dimmable LED Indoor/Outdoor Black...,/p/CARRO-Aspen-52-in-Dimmable-LED-Indoor-Outdo...,318630540,319131133,S525J1-L11-B2-1G,169.99,299.0,117.0,True,29,3.8966
3,4,319131207,HS525J1-L11-W1-1,1007081141,CARRO,Aspen 52 in. Dimmable LED Indoor/Outdoor White...,/p/CARRO-Aspen-52-in-Dimmable-LED-Indoor-Outdo...,319131207,319131133,HS525J1-L11-W1-1,169.99,299.0,86.0,True,29,3.8966
4,5,322225567,HS565J1-L11-W1-1,None,None,None,None,322225567,None,None,NaN,NaN,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,603,315651618,PH-E061008-2E1,1006008508,CARRO,Nature Collection Limited Edition Round Empire...,/p/CARRO-Nature-Collection-Limited-Edition-Rou...,315651618,315598906,PH-E061008-2E1,12.90,NaN,NaN,None,2,5.0
603,604,315643876,PH-E071308-2H1,1006007205,CARRO,Floral Collection Limited Edition Round Empire...,/p/CARRO-Floral-Collection-Limited-Edition-Rou...,315643876,315598911,PH-E071308-2H1,15.90,NaN,NaN,None,5,5.0
604,605,323684397,H-UC9032,1009070140,CARRO,"Pyatt 3-Speed Slide Ceiling Fan Wall Switch, L...",/p/CARRO-Pyatt-3-Speed-Slide-Ceiling-Fan-Wall-...,323684397,None,H-UC9032,34.24,NaN,32.0,True,0,None
605,606,315277027,PN-04F01A-WH1,1005927564,CARRO,Pioneer Smart Wi-Fi Ceiling Fan Wall Switch (1...,/p/CARRO-Pioneer-Smart-Wi-Fi-Ceiling-Fan-Wall-...,315277027,None,PN-04F01A-WH1,47.04,NaN,NaN,None,12,4.0833


In [100]:
df_merge.to_excel("Carro前台所有吊扇信息爬取表(9.13).xlsx", index=False)